# Contoh penggunaan `morph_analyzer.py`

Pakej yang diperlukan: `pyspellchecker` (Dapatkan di https://pypi.org/project/pyspellchecker/ jika tidak termasuk dalam sistem anda.)

(Required package: `pyspellchecker` (Download from https://pypi.org/project/pyspellchecker/ if your system does not have it.))

In [23]:
pip install pyspellchecker

You should consider upgrading via the 'c:\Users\ACER\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [37]:
pip install nlp-id

^C
Note: you may need to restart the kernel to use updated packages.


In [2]:
# from spellchecker import SpellChecker
import pickle 
with open("rootlist.pkl", "rb") as f:
    rootlist = pickle.load(f)
import morph_analyzer as ma

## Tanpa kamus MALINDO Morph

(Without the MALINDO Morph dictionary)

In [11]:
ma.morph("keberkaitananlah", rootlist)

{('kait', 'keberkaitananlah', '0', '-lah', 'ber--an+ke--an', '0'),
 ('kait', 'keberkaitananlah', 'ber-', '-an+-lah', 'ke--an', '0'),
 ('kait', 'keberkaitananlah', 'ke-', '-an+-lah', 'ber--an', '0')}

### Parameter `Indo`

Parameter `Indo` mengaktifkan awalan _N-_ (cth. _N-_ + _kopi_ = _ngopi_).

(The `Indo` parameter activates the prefix _N-_ (e.g. _N-_ + _kopi_ = _ngopi_).)

In [12]:
ma.morph("nyampai", rootlist)

{('nyampa', 'nyampai', '0', '-i', '0', '0')}

In [13]:
ma.morph("nyampai", rootlist, Indo=True)

{('campa', 'nyampai', 'N-', '-i', '0', '0'),
 ('sampai', 'nyampai', 'N-', '0', '0', '0')}

### Parameter `n`

Parameter `n` mengawal jumlah calon yang dihasilkan.  Nilai lalainya ialah 5.

(The `n` parameter controls the number of the candidates to be produced.  The default value is 5.)

In [14]:
ma.morph("mengebibkah", rootlist)

{('bibkah', 'mengebibkah', 'ke-+meN-', '0', '0', '0'),
 ('ebib', 'mengebibkah', 'meN-', '-kah', '0', '0'),
 ('ebibkah', 'mengebibkah', 'meN-', '0', '0', '0'),
 ('kebib', 'mengebibkah', 'meN-', '-kah', '0', '0'),
 ('ngebib', 'mengebibkah', 'meN-', '-kah', '0', '0')}

In [15]:
ma.morph("mengebibkah", rootlist, n=15)

{('bib', 'mengebibkah', 'meN-', '-kah', '0', '0'),
 ('bibkah', 'mengebibkah', 'ke-+meN-', '0', '0', '0'),
 ('ebib', 'mengebibkah', 'meN-', '-kah', '0', '0'),
 ('ebibkah', 'mengebibkah', 'meN-', '0', '0', '0'),
 ('kebib', 'mengebibkah', 'meN-', '-kah', '0', '0'),
 ('kebibkah', 'mengebibkah', 'meN-', '0', '0', '0'),
 ('mengebib', 'mengebibkah', '0', '-kah', '0', '0'),
 ('mengebibkah', 'mengebibkah', '0', '0', '0', '0'),
 ('ngebib', 'mengebibkah', 'meN-', '-kah', '0', '0'),
 ('ngebibkah', 'mengebibkah', 'meN-', '0', '0', '0')}

## Bersama dengan kamus MALINDO Morph

Walaupun `morph_analzser.py` boleh digunakan secara sendirian, adalah lebih realistik untuk menggunakannya bersama dengan kamus MALINDO Morph yang (kebanyakan) analisis morfologinya sudah diperiksa oleh manusia.  Dalam contoh kod di bawah, `morph_analyzer.py` digunakan hanya apabila perkataan yang ingin dianalisis tidak termasuk dalam kamus MALINDO Morph.

(With the MALINDO Morph dictionary  
Although `morph_analyzer.py` can be used by itself, it is more realistic to use it with the MALINDO Morph dictionary in which most of the morphological analyses have been examined manually.  In the code below, `morph_analyzer.py` is used only when the word to be analysed is not found in the MALINDO Morph dictionary.)

In [3]:
# Buat kamus daripada MALINDO Morph
with open("malindo_dic_20200917.tsv", "r", encoding="utf-8") as f: #Gunakan versi terkini MALINDO Moprh
    katakata = []
    for l in f:
        items = l.strip().split("\t")
        if not items[0].startswith("ex-"): #bahagian yg sudah diperiksa manusia sahaja
            katakata.append(tuple(items[1:7])) #tanpa ID, sumber, dasar, lema 

kamus = dict()
for kata in katakata:
    surface = kata[1]
    if not surface in kamus.keys():
        kamus[surface] = []
    kamus[surface].append(kata)

In [63]:
def analisis(w, Indo=True, n=5):
    try:
        return kamus[w][:n]
    except:
        return list(ma.morph(w, rootlist, Indo, n))

### Perkataan yang ada dalam kamus MALINDO Morph

(Words available in the MALINDO Morph dictionary)

In [64]:
KJ = "Flowchart adalah bagan yang menunjukkan arus pekerjaan secara keseluruhan dari sistem"
jawaban = "Bagan yang menunjukkan alur menyeluruh kerja sistem"

In [65]:
lower_kj = KJ.lower()
lower_js = jawaban.lower()
print(lower_kj)
print(lower_js)

flowchart adalah bagan yang menunjukkan arus pekerjaan secara keseluruhan dari sistem
bagan yang menunjukkan alur menyeluruh kerja sistem


In [66]:
token1 = lower_kj.split()
token2 = lower_js.split()
print(token1)
print(token2)

['flowchart', 'adalah', 'bagan', 'yang', 'menunjukkan', 'arus', 'pekerjaan', 'secara', 'keseluruhan', 'dari', 'sistem']
['bagan', 'yang', 'menunjukkan', 'alur', 'menyeluruh', 'kerja', 'sistem']


In [119]:
lemma = []
hasil_lemma = []
for token in token1 :
    lemma = analisis(token)
    # print(lemma)
    # print(lemma[0][0])
    hasil_lemma.append(lemma[0][0])
print("Hasil lemma : ", hasil_lemma)

Hasil lemma :  ['flowchart', 'ada', 'bagan', 'yang', 'tunjuk', 'arus', 'kerja', 'cara', 'seluruh', 'dari', 'sistem']


In [32]:
analisis("keseluruhan")

[('seluruh', 'keseluruhan', '0', '0', 'ke--an', '0')]

In [19]:
analisis("mereka")

[('mereka', 'mereka', '0', '0', '0', '0'),
 ('reka', 'mereka', 'meN-', '0', '0', '0')]

In [20]:
analisis("mereka", n=1)

[('mereka', 'mereka', '0', '0', '0', '0')]

### Perkataan yang tidak ada dalam kamus MALINDO Morph

(Words unavailable in the MALINDO Morph dictionary)

In [21]:
analisis("mengepobkan")

[('ngepob', 'mengepobkan', 'meN-', '-kan', '0', '0'),
 ('pobkan', 'mengepobkan', 'ke-+meN-', '0', '0', '0'),
 ('epob', 'mengepobkan', 'meN-', '-kan', '0', '0'),
 ('epobk', 'mengepobkan', 'meN-', '-an', '0', '0'),
 ('kepob', 'mengepobkan', 'meN-', '-kan', '0', '0')]

In [22]:
analisis("mengepobkan", n=3)

[('epobk', 'mengepobkan', 'meN-', '-an', '0', '0'),
 ('kepob', 'mengepobkan', 'meN-', '-kan', '0', '0'),
 ('epob', 'mengepobkan', 'meN-', '-kan', '0', '0')]

In [23]:
analisis("terjadi berulang-ulang")

[('terjadi berulang-ulang', 'terjadi berulang-ulang', '0', '0', '0', '0')]